# @classmethod ポリモフィズムを使ってオブジェクトをジェネリックに構築する

Python がオブジェクトだけでなくクラスやポリモフィズムをサポートする意味

ポリモフィズムを使用すると、階層をなす複数クラスでそれぞれ独自のバージョンのメソッドを実装できる

この方式では、多くのクラスが同じインタフェース、あるいは、抽象基底クラスを実現しながら、
異なった機能を提供する

In [ ]:
# MapReduce の実装を書いていて、入力データを表す共通クラスが欲しいとする
# サブクラスで定義する必要のある read メソッドを持つクラスを次のように定義
# 抽象クラス
class InputData:
  def read(self):
    raise NotImplementedError

In [ ]:
# データをディスクのファイルから読み込む InputData の具象サブクラス
class PathInputData(InputData):
  def __init__(self, path):
    super().__init__()
    self.path = path

  def read(self):
    with open(self.path) as f:
      return f.read()

In [ ]:
# 抽象クラス
class Worker:
  def __init__(self, input_data):
    self.input_data = input_data
    self.result = None

  def map(self):
    raise NotImplementedError
  
  def reduce(self, other):
    raise NotImplementedError

In [4]:
class LineCountWorker(Worker):
  def map(self):
    data = self.input_data.read()
    self.result = data.count('\n')

  def reduce(self, other):
    self.result += other.result

In [5]:
import os

def generate_inputs(data_dir):
  for name in os.listdir(data_dir):
    yield PathInputData(os.path.join(data_dir, name))

In [6]:
def create_workers(input_list):
  workers = []
  for input_data in input_list:
    workers.append(LineCountWorker(input_data))
  return workers

In [7]:
from threading import Thread

def execute(workers):
  threads = [Thread(target=w.map) for w in workers]
  for thread in threads: thread.start()
  for thread in threads: thread.join()

  first, *rest = workers
  for worker in rest:
    first.reduce(worker)
  return first.result

In [8]:
def mapreduce(data_dir):
  inputs = generate_inputs(data_dir)
  workers = create_workers(inputs)
  return execute(workers)

In [ ]:
import os
import random

def write_test_files(tmpdir):
  os.makedirs(tmpdir)
  for i in range(100):
    with open(os.path.join(tmpdir, str(i)), 'w') as f:
      f.write('\n' * random.randint(0, 100))

tmpdir = 'test_inputs'
write_test_files(tmpdir)

result = mapreduce(tmpdir)
print(f'There are {result} lines')

There are 4704 lines


これの大きな問題は、この mapreduce関数がまったくジェネリックではないことである
つまり、mapreduce 関数が汎用的（ジェネリック）ではなく、特定の具象クラス（PathInputData, LineCountWorker）に依存してしまっているという点が問題

### 問題の背景

このコードの構造は、以下のような意図で設計されている

- InputData クラス：データの読み込み方法を抽象化（抽象クラス）
- Worker クラス：MapReduce ワーカーの抽象クラス
- それぞれに対して、具象クラス（PathInputData, LineCountWorker）が存在

つまり、ポリモーフィズム（多態性）を活かして、他のタイプの InputData（たとえばネットワーク経由でデータを取得する NetworkInputData）や、他のタイプの Worker（たとえば単語数を数える WordCountWorker）を定義できる構造にしてある

しかし、generate_inputs関数やcreate_workers関数の中で、具体的なクラス（PathInputDataとLineCountWorker）が固定的に使われている

```Python
def generate_inputs(data_dir):
  for name in os.listdir(data_dir):
    yield PathInputData(os.path.join(data_dir, name))
```

```Python
def create_workers(input_list):
  workers = []
  for input_data in input_list:
    workers.append(LineCountWorker(input_data))
  return workers
```

そのため、以下のことを考えた際に、mapreduce 関数やその周辺（generate_inputs, create_workers）を書き換える必要がある

- 他の InputData のサブクラス（たとえば MemoryInputData）を使いたい
- 他の Worker のサブクラス（たとえば WordCountWorker）を使いたい

```Python
def mapreduce(data_dir):
  inputs = generate_inputs(data_dir) # ← PathInputDataに固定
  workers = create_workers(inputs)   # ← LineCountWorkerに固定
  return execute(workers)
```

このように、クラス構成自体はポリモーフィズムに対応していて柔軟に見えるのに、実際の使用箇所ではクラス名がベタ書きされているために柔軟性を失っている——これが「mapreduce関数がジェネリックではない」という意味です。

### 問題の解決法
別の InputData や Worker といったサブクラスを書いたなら generate_inputs や create_workers を書き直して、mapreduce 関数でも対応しなければいけない

この問題を解く最良の方法は、クラスメソッドポリモフィズムを使う

In [15]:
class GenericInputData:
  def read(self):
    raise NotImplementedError
  
  @classmethod
  def generate_inputs(cls, config):
    raise NotImplementedError

In [25]:
class PathInputData(GenericInputData):
  def __init__(self, path):
    super().__init__()
    self.path = path

  def read(self):
    with open(self.path) as f:
      return f.read()
  
  @classmethod
  def generate_inputs(cls, config):
    data_dir = config['data_dir']
    for name in os.listdir(data_dir):
      yield cls(os.path.join(data_dir, name))

In [24]:
class GenericWorker:
  def __init__(self, input_data):
    self.input_data = input_data
    self.result = None

  def map(self):
    raise NotImplementedError
  
  def reduce(self, other):
    raise NotImplementedError
  
  @classmethod
  def create_workers(cls, input_class, config):
    workers = []
    for input_data in input_class.generate_inputs(config):
      workers.append(cls(input_data))
    return workers

In [18]:
class LineCountWorker(GenericWorker):
  def map(self):
    data = self.input_data.read()
    self.result = data.count('\n')

  def reduce(self, other):
    self.result += other.result

In [19]:
def mapreduce(worker_class, input_class, config):
  workers = worker_class.create_workers(input_class, config)
  return execute(workers)

In [26]:
config = {'data_dir': tmpdir}
result = mapreduce(LineCountWorker, PathInputData, config)
print(f'There are {result} lines')

There are 4704 lines


以下のように書くこともできる

```Python
mapreduce(WordCountWorker, NetworkInputData, 'http://example.com/data')
```